In [102]:
import numpy as np
import pandas as pd
from math import ceil
import dateutil
import pickle
with open('datapath.txt') as f:
    datapath=f.readlines()[0].rstrip()



In [2]:
import bisect

In [4]:
totaltrainrows=37670293
ndestinations=62106
totaltestrows=2528243

In [5]:
categorical=[u'site_name', u'posa_continent', u'user_location_country',
       u'user_location_region', u'user_location_city',u'user_id', u'is_mobile', u'is_package',
       u'channel',u'srch_destination_id',
       u'srch_destination_type_id', u'is_booking',u'hotel_continent',
       u'hotel_country', u'hotel_market', u'hotel_cluster']

In [67]:
integers=[u'srch_adults_cnt',
       u'srch_children_cnt', u'srch_rm_cnt','cnt']

In [7]:
numbers=[u'srch_adults_cnt',
       u'srch_children_cnt', u'srch_rm_cnt','orig_destination_distance','cnt']

In [8]:
dates=[u'srch_ci', u'srch_co','date_time']

In [68]:
nchunks=20
chunksize=ceil(totaltrainrows/nchunks)
trainit=pd.read_csv(datapath+'train.csv',chunksize=chunksize,iterator=True,usecols=categorical+integers)

In [69]:
uniques={i:set() for i in categorical+integers}

In [70]:
def checkuniques(x):
    uniques[x.name].update(set(x.unique().tolist()))

In [71]:
%%time
for chunk in trainit:
    chunk.apply(checkuniques)  

CPU times: user 1min 5s, sys: 12.3 s, total: 1min 18s
Wall time: 1min 30s


In [50]:
{i:(any(map(np.isnan,uniques[i]))) for i in uniques.keys()}

{u'channel': False,
 u'hotel_cluster': False,
 u'hotel_continent': False,
 u'hotel_country': False,
 u'hotel_market': False,
 u'is_booking': False,
 u'is_mobile': False,
 u'is_package': False,
 u'posa_continent': False,
 u'site_name': False,
 u'srch_adults_cnt': False,
 u'srch_children_cnt': False,
 u'srch_destination_id': False,
 u'srch_destination_type_id': False,
 u'srch_rm_cnt': False,
 u'user_id': False,
 u'user_location_city': False,
 u'user_location_country': False,
 u'user_location_region': False}

In [72]:
dfdict={i:[min(uniques[i]), max(uniques[i]),len(uniques[i])] for i in uniques.keys()}

In [73]:
featureranges=pd.DataFrame(dfdict.values(),index=dfdict.keys(),columns=['min','max','n_uniques'])

In [74]:
featureranges

,min,max,n_uniques
hotel_continent,0,6,7
srch_children_cnt,0,9,10
srch_destination_id,0,65107,59455
user_id,0,1198785,1198786
user_location_country,0,239,237
is_mobile,0,1,2
is_booking,0,1,2
cnt,1,269,104
srch_adults_cnt,0,9,10
hotel_country,0,212,213


In [75]:
uintypes=[np.uint8,np.uint16,np.uint32]

In [76]:
uintmax={np.iinfo(i).max:i for i in uintypes}

In [77]:
uintmax[1]=np.bool_

In [78]:
def find_gt(a, x):
    'Find leftmost value greater than or equal to x'
    i = bisect.bisect_left(a, x)
    if i != len(a):
        return a[i]
    raise ValueError
    

In [79]:
keys=sorted(uintmax.keys())

In [80]:
featuretypes=featureranges['max'].map(lambda x: uintmax[find_gt(keys,x)]).to_dict()

In [81]:
featuretypes

{u'channel': numpy.uint8,
 'cnt': numpy.uint16,
 u'hotel_cluster': numpy.uint8,
 u'hotel_continent': numpy.uint8,
 u'hotel_country': numpy.uint8,
 u'hotel_market': numpy.uint16,
 u'is_booking': numpy.bool_,
 u'is_mobile': numpy.bool_,
 u'is_package': numpy.bool_,
 u'posa_continent': numpy.uint8,
 u'site_name': numpy.uint8,
 u'srch_adults_cnt': numpy.uint8,
 u'srch_children_cnt': numpy.uint8,
 u'srch_destination_id': numpy.uint16,
 u'srch_destination_type_id': numpy.uint8,
 u'srch_rm_cnt': numpy.uint8,
 u'user_id': numpy.uint32,
 u'user_location_city': numpy.uint16,
 u'user_location_country': numpy.uint8,
 u'user_location_region': numpy.uint16}

In [98]:
traindata=pd.read_csv(datapath+'train.csv',nrows=100,dtype=featuretypes,parse_dates=dates)

In [115]:
traindata.dtypes

date_time                    datetime64[ns]
site_name                             uint8
posa_continent                        uint8
user_location_country                 uint8
user_location_region                 uint16
user_location_city                   uint16
orig_destination_distance           float64
user_id                              uint32
is_mobile                              bool
is_package                             bool
channel                               uint8
srch_ci                      datetime64[ns]
srch_co                      datetime64[ns]
srch_adults_cnt                       uint8
srch_children_cnt                     uint8
srch_rm_cnt                           uint8
srch_destination_id                  uint16
srch_destination_type_id              uint8
is_booking                             bool
cnt                                  uint16
hotel_continent                       uint8
hotel_country                         uint8
hotel_market                    

In [112]:
with open('feature_dtypes.pkl', 'w') as f:
    pickle.dump(featuretypes,f)

In [113]:
with open('feature_dtypes.pkl', 'r') as f:
    featurefromfile=pickle.load(f)

In [114]:
pd.read_csv(datapath+'train.csv',nrows=100,dtype=featurefromfile,parse_dates=dates)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,False,True,...,0,1,8250,1,False,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,False,True,...,0,1,8250,1,True,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,False,False,...,0,1,8250,1,False,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,False,False,...,0,1,14984,1,False,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,False,False,...,0,1,14984,1,False,1,2,50,1457,21
5,2014-08-09 18:13:12,2,3,66,442,35390,911.5142,93,False,False,...,0,1,14984,1,False,1,2,50,1457,92
6,2014-07-16 09:42:23,2,3,66,189,10067,NaN,501,False,False,...,0,1,8267,1,False,2,2,50,675,41
7,2014-07-16 09:45:48,2,3,66,189,10067,NaN,501,False,True,...,0,1,8267,1,False,1,2,50,675,41
8,2014-07-16 09:52:11,2,3,66,189,10067,NaN,501,False,False,...,0,1,8267,1,False,1,2,50,675,69
9,2014-07-16 09:55:24,2,3,66,189,10067,NaN,501,False,False,...,0,1,8267,1,False,1,2,50,675,70
